# Histopathologic Cancer Classification

## Loading libraries and data

In [10]:
import torch                # I have not used keras much before. I'm more familiar with pytorch so I'll use that
import tifffile as tiff     # For reading in the .tif images
import pandas as pd
import matplotlib.pyplot as plt
import os

# Want to make sure I'm training on my GPU because I do not have one million years to train on my CPU
torch.set_default_device('cuda')
torch.cuda.get_device_name(0)

train_labels = pd.read_csv('train_labels.csv')
train_labels.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


## EDA

In [ ]:
# Checking for class imbalance in training data labels
print(train_labels['label'].value_counts())
duplicate_ids = train_labels.duplicated(subset='id').sum()
print(f'Found {duplicate_ids} duplicated file IDs')

# Test image reading
def show_image(image_id: str) -> None:
    path = os.path.join('train', f'{image_id}.tif')
    img = tiff.imread(path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(image_id)
    plt.show()

sample_images = train_labels.sample(n=1000)

show_image(sample_images.iloc[100]['id'])

# Check for corrupt or files not matching expectations
bad_files = []
for image_id in sample_images['id']:
    try:
        img = tiff.imread(os.path.join('train', f'{image_id}.tif'))
        assert img.shape == (96, 96, 3)
    except Exception as e:
        print(f'image {image_id} does not match format expectations.')
        bad_files.append(image_id)

print(f'Found {len(bad_files)} images not matching format expectations')






del sample_images


0


## Model building

## Train Data Evaluation